# Import Libraries

In [2]:
import pandas as pd

# Obtain Data

In [12]:
df = pd.read_csv('Data/census_data/census_data.csv')
df.head()

,GEO_ID,NAME,S0701_C01_001E,S0701_C01_001M,S0701_C01_002E,S0701_C01_002M,S0701_C01_003E,S0701_C01_003M,S0701_C01_004E,S0701_C01_004M,...,S0701_C05_052E,S0701_C05_052M,S0701_C05_053E,S0701_C05_053M,S0701_C05_054E,S0701_C05_054M,S0701_C05_055E,S0701_C05_055M,S0701_C05_056E,S0701_C05_056M
0,id,Geographic Area Name,Estimate!!Total!!Population 1 year and over,Margin of Error!!Total!!Population 1 year and ...,Estimate!!Total!!Population 1 year and over!!A...,Margin of Error!!Total!!Population 1 year and ...,Estimate!!Total!!Population 1 year and over!!A...,Margin of Error!!Total!!Population 1 year and ...,Estimate!!Total!!Population 1 year and over!!A...,Margin of Error!!Total!!Population 1 year and ...,...,Estimate!!Moved; from abroad!!POVERTY STATUS I...,Margin of Error!!Moved; from abroad!!POVERTY S...,Estimate!!Moved; from abroad!!HOUSING TENURE!!...,Margin of Error!!Moved; from abroad!!HOUSING T...,Estimate!!Moved; from abroad!!HOUSING TENURE!!...,Margin of Error!!Moved; from abroad!!HOUSING T...,Estimate!!Moved; from abroad!!HOUSING TENURE!!...,Margin of Error!!Moved; from abroad!!HOUSING T...,Estimate!!Moved; from abroad!!PERCENT ALLOCATE...,Margin of Error!!Moved; from abroad!!PERCENT A...
1,0500000US01003,"Baldwin County, Alabama",221737,907,9119,1103,36287,926,16614,1496,...,0.1,0.1,0.2,0.2,0.1,0.1,0.7,1.1,(X),(X)
2,0500000US01015,"Calhoun County, Alabama",112208,658,5302,657,17775,200,10047,750,...,0.0,0.2,0.0,0.2,0.0,0.2,0.0,0.5,(X),(X)
3,0500000US01043,"Cullman County, Alabama",83222,346,4764,706,13503,609,6160,397,...,0.0,0.3,0.0,0.2,0.0,0.3,0.0,0.9,(X),(X)
4,0500000US01049,"DeKalb County, Alabama",N,N,N,N,N,N,N,N,...,N,N,0.1,0.1,0.1,0.1,0.1,0.1,(X),(X)


In [8]:
! ls Data

census_data.csv
